In [1]:
!pip install -qU awscli boto3 sagemaker

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
ALGORITHM_NAME = 'multi-model-sklearn-1'

In [3]:
%%sh -s $ALGORITHM_NAME

algorithm_name=$1

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

ecr_image="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email --registry-ids ${account})

# Build the docker image locally with the image name and then push it to ECR
# with the full image name.

# First clear out any prior version of the cloned repo
rm -rf sagemaker-scikit-learn-container/

# Clone the sklearn container repo
git clone --single-branch --branch mme https://github.com/aws/sagemaker-scikit-learn-container.git
cd sagemaker-scikit-learn-container/
# echo "s3fs" >> requirements.txt

# Build the "base" container image that encompasses the installation of the
# scikit-learn framework and all of the dependencies needed.
docker build -q -t sklearn-base:0.20-2-cpu-py3 -f docker/0.20-2/base/Dockerfile.cpu --build-arg py_version=3 .

# Create the SageMaker Scikit-learn Container Python package.
python setup.py bdist_wheel --universal

# Build the "final" container image that encompasses the installation of the
# code that implements the SageMaker multi-model container requirements.
docker build -t ${algorithm_name} -f docker/0.20-2/final/Dockerfile.cpu .

docker tag ${algorithm_name} ${ecr_image}

docker push ${ecr_image}

Login Succeeded
sha256:802d9f5df0911ab65a5130cd92bacf0083e55a7c321b5ecb127104e78ee8c7d9
running bdist_wheel
running build
running build_py
creating build
creating build/lib
creating build/lib/sagemaker_sklearn_container
copying src/sagemaker_sklearn_container/serving.py -> build/lib/sagemaker_sklearn_container
copying src/sagemaker_sklearn_container/__init__.py -> build/lib/sagemaker_sklearn_container
copying src/sagemaker_sklearn_container/training.py -> build/lib/sagemaker_sklearn_container
copying src/sagemaker_sklearn_container/handler_service.py -> build/lib/sagemaker_sklearn_container
creating build/lib/sagemaker_sklearn_container/mms_patch
copying src/sagemaker_sklearn_container/mms_patch/__init__.py -> build/lib/sagemaker_sklearn_container/mms_patch
copying src/sagemaker_sklearn_container/mms_patch/model_server.py -> build/lib/sagemaker_sklearn_container/mms_patch
copying src/sagemaker_sklearn_container/mms_patch/mms_transformer.py -> build/lib/sagemaker_sklearn_container/mms_p

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Cloning into 'sagemaker-scikit-learn-container'...


In [6]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import boto3
import os
sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()
# ACCOUNT_ID  = boto3.client('sts').get_caller_identity()['Account']
# REGION      = boto3.Session().region_name
# MULTI_MODEL_SKLEARN_IMAGE = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(ACCOUNT_ID, REGION, 
#                                                                            ALGORITHM_NAME)

ALGORITHM_NAME = 'multi-model-sklearn-1'
ACCOUNT_ID  = boto3.client('sts').get_caller_identity()['Account']
REGION      = boto3.Session().region_name
# BUCKET      = sagemaker_session.default_bucket()
SCRIPT_FILENAME     = 'nbr_spot_model.py'
USER_CODE_ARTIFACTS = 'user_code.tar.gz'

MULTI_MODEL_SKLEARN_IMAGE = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(ACCOUNT_ID, REGION, 
                                                                           ALGORITHM_NAME)


TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'

In [7]:
MULTI_MODEL_SKLEARN_IMAGE

'738774084799.dkr.ecr.us-east-1.amazonaws.com/multi-model-sklearn-1:latest'